In [ ]:
#Script to compile index list and generate daily IRS report
import datetime
import requests
import os.path
import os
from zipfile import ZipFile
import csv
import psycopg2
import pandas as pd
import calendar
import numpy as np
import pandas.io.sql as sqlio
import time
import math
from datetime import timedelta
from utils.db_helper import DB_Helper
import utils.date_set as date_set

In [ ]:
conn = DB_Helper().db_connect()
cur = conn.cursor()
dbURL = DB_Helper().engine()

In [16]:
def gen_sub_industry_index(conn, date):
    """ Generate the Industry index

    Operation:
        Fetch the data from IndustryDivisor and IndustryList for current date,
        and calculate the Open, close, high, low, PE , EPS and Earning Growth,

        'open' = ff_open/divisor, 'high' = ff_high/divisor, 'low' = ff_low/divisor,

        'close' = ff_close/divisor, 'earnings growth' = (eps – eps back / eps back) * 100 ,

        'pe' = sum os close / ff earning, eps = ff earning / ff equity.

    Return:
        Data of Industry index.
    """
      # Generating Industry index list

    industry_divisor_sql = 'SELECT * FROM public."SubIndustryDivisor" WHERE "Date" = \''+date+'\' ;'
    industry_divisor = sqlio.read_sql_query(industry_divisor_sql, con = conn)

    industry_ff_ohlc_sql = 'SELECT "SubIndustryIndexName", "SubIndustry", \
                        SUM("OS_Close") AS sum_os_close,\
                        SUM("FF_Open") AS ff_open_sum, \
                        SUM("FF_High") AS ff_high_sum, \
                        SUM("FF_Low") AS ff_low_sum, \
                        SUM("FF_Close") AS ff_close_sum, \
                        SUM("Volume") AS sum_vol, \
                        SUM("OS") AS os_sum, \
                        SUM("PAT") AS earnings, \
                        SUM("Equity") AS sum_equity,   \
                        SUM("prev_pat") AS prev_earnings, \
                        SUM("prev_equity") AS prev_equity, \
                        count(*) AS company_count\
                        FROM public."IndustryList" \
                        WHERE "GenDate" = \''+date+'\' AND "SubIndustryIndexName" is not null \
                        GROUP BY "SubIndustryIndexName", "SubIndustry" ;'
    industry_ff_ohlc = sqlio.read_sql_query(industry_ff_ohlc_sql, con = conn)

    for index, row in industry_ff_ohlc.iterrows():

      ff_open_list = industry_ff_ohlc.loc[(industry_ff_ohlc['SubIndustryIndexName']==row['SubIndustryIndexName'])]['ff_open_sum']
      ff_open = ff_open_list.item() if len(ff_open_list.index) == 1 else np.nan

      ff_high_list = industry_ff_ohlc.loc[(industry_ff_ohlc['SubIndustryIndexName']==row['SubIndustryIndexName'])]['ff_high_sum']
      ff_high = ff_high_list.item() if len(ff_high_list.index) == 1 else np.nan

      ff_low_list = industry_ff_ohlc.loc[(industry_ff_ohlc['SubIndustryIndexName']==row['SubIndustryIndexName'])]['ff_low_sum']
      ff_low = ff_low_list.item() if len(ff_low_list.index) == 1 else np.nan

      ff_close_list = industry_ff_ohlc.loc[(industry_ff_ohlc['SubIndustryIndexName']==row['SubIndustryIndexName'])]['ff_close_sum']
      ff_close = ff_close_list.item() if len(ff_close_list.index) == 1 else np.nan

      ff_os_list = industry_ff_ohlc.loc[(industry_ff_ohlc['SubIndustryIndexName']==row['SubIndustryIndexName'])]['os_sum']
      ff_os = ff_os_list.item() if len(ff_os_list.index) == 1 else np.nan

      ff_earnings_list = industry_ff_ohlc.loc[(industry_ff_ohlc['SubIndustryIndexName']==row['SubIndustryIndexName'])]['earnings']
      ff_earnings = ff_earnings_list.item() if len(ff_earnings_list.index) == 1 else np.nan

      ff_equity_list = industry_ff_ohlc.loc[(industry_ff_ohlc['SubIndustryIndexName']==row['SubIndustryIndexName'])]['sum_equity']
      ff_equity = ff_equity_list.item() if len(ff_equity_list.index) == 1 else np.nan

      prev_earnings_list = industry_ff_ohlc.loc[(industry_ff_ohlc['SubIndustryIndexName']==row['SubIndustryIndexName'])]['prev_earnings']
      prev_earnings = prev_earnings_list.item() if len(prev_earnings_list.index) == 1 else np.nan

      prev_equity_list = industry_ff_ohlc.loc[(industry_ff_ohlc['SubIndustryIndexName']==row['SubIndustryIndexName'])]['prev_equity']
      prev_equity = prev_equity_list.item() if len(prev_equity_list.index) == 1 else np.nan

      divisor_list = industry_divisor.loc[(industry_divisor['IndexName']==row['SubIndustryIndexName'])]['Divisor']
      divisor = divisor_list.item() if len(divisor_list.index) == 1 else np.nan

      sum_os_close_list = industry_ff_ohlc.loc[(industry_ff_ohlc['SubIndustryIndexName']==row['SubIndustryIndexName'])]['sum_os_close']
      sum_os_close = sum_os_close_list.item() if len(sum_os_close_list.index) == 1 else np.nan

      if np.isnan(divisor) or divisor == 0:
        divisor  =  10000
        print("Devisor gen_Subindustry_index: ", divisor)

      div_open = ff_open/divisor if divisor != 0 and not np.isnan(ff_open) else np.nan
      div_high = ff_high/divisor if divisor != 0 and not np.isnan(ff_high) else np.nan
      div_low = ff_low/divisor if divisor != 0 and not np.isnan(ff_low) else np.nan
      div_close = ff_close/divisor if divisor != 0 and not np.isnan(ff_close) else np.nan


      # div_open = ff_open/divisor if divisor != 0 and not np.isnan(ff_open) else np.nan
      # div_open = ff_open/divisor if divisor and ff_open is not None and divisor != 0 else np.nan
      # div_high = ff_high/divisor if divisor and ff_high is not None and divisor != 0 else np.nan
      # div_low = ff_low/divisor if divisor and ff_low is not None and divisor != 0 else np.nan
      # div_close = ff_close/divisor if divisor and ff_close is not None and divisor != 0 else np.nan


      pe = (sum_os_close)/ff_earnings if ff_earnings !=0 else np.nan
      eps = ff_earnings/ff_equity if ff_equity !=0 else np.nan
      eps_back = prev_earnings/prev_equity if prev_equity !=0 else np.nan
      earnings_growth = ((eps-eps_back)/abs(eps_back))*100 if eps_back != 0 else np.nan


      industry_ff_ohlc.loc[index, 'Open'] = div_open
      industry_ff_ohlc.loc[index, 'High'] = div_high
      industry_ff_ohlc.loc[index, 'Low'] = div_low
      industry_ff_ohlc.loc[index, 'Close'] = div_close
      industry_ff_ohlc.loc[index, 'PE'] = pe
      industry_ff_ohlc.loc[index, 'EPS'] = eps
      industry_ff_ohlc.loc[index, 'Earnings Growth'] = earnings_growth


    return industry_ff_ohlc

In [3]:
def gen_industry_index(conn, date):
    """ Generate the Industry index

    Operation:
        Fetch the data from IndustryDivisor and IndustryList for current date,
        and calculate the Open, close, high, low, PE , EPS and Earning Growth,

        'open' = ff_open/divisor, 'high' = ff_high/divisor, 'low' = ff_low/divisor,

        'close' = ff_close/divisor, 'earnings growth' = (eps – eps back / eps back) * 100 ,

        'pe' = sum os close / ff earning, eps = ff earning / ff equity.

    Return:
        Data of Industry index.
    """
      # Generating Industry index list

    industry_divisor_sql = 'SELECT * FROM public."IndustryDivisor" WHERE "Date" = \''+date+'\' ;'
    industry_divisor = sqlio.read_sql_query(industry_divisor_sql, con = conn)

    industry_ff_ohlc_sql = 'SELECT "IndustryIndexName", "Industry", \
                        SUM("OS_Close") AS sum_os_close,\
                        SUM("FF_Open") AS ff_open_sum, \
                        SUM("FF_High") AS ff_high_sum, \
                        SUM("FF_Low") AS ff_low_sum, \
                        SUM("FF_Close") AS ff_close_sum, \
                        SUM("Volume") AS sum_vol, \
                        SUM("OS") AS os_sum, \
                        SUM("PAT") AS earnings, \
                        SUM("Equity") AS sum_equity,   \
                        SUM("prev_pat") AS prev_earnings, \
                        SUM("prev_equity") AS prev_equity, \
                        count(*) AS company_count\
                        FROM public."IndustryList" \
                        WHERE "GenDate" = \''+date+'\' AND "IndustryIndexName" is not null \
                        GROUP BY "IndustryIndexName", "Industry" ;'
    industry_ff_ohlc = sqlio.read_sql_query(industry_ff_ohlc_sql, con = conn)

    for index, row in industry_ff_ohlc.iterrows():

      ff_open_list = industry_ff_ohlc.loc[(industry_ff_ohlc['IndustryIndexName']==row['IndustryIndexName'])]['ff_open_sum']
      ff_open = ff_open_list.item() if len(ff_open_list.index) == 1 else np.nan

      ff_high_list = industry_ff_ohlc.loc[(industry_ff_ohlc['IndustryIndexName']==row['IndustryIndexName'])]['ff_high_sum']
      ff_high = ff_high_list.item() if len(ff_high_list.index) == 1 else np.nan

      ff_low_list = industry_ff_ohlc.loc[(industry_ff_ohlc['IndustryIndexName']==row['IndustryIndexName'])]['ff_low_sum']
      ff_low = ff_low_list.item() if len(ff_low_list.index) == 1 else np.nan

      ff_close_list = industry_ff_ohlc.loc[(industry_ff_ohlc['IndustryIndexName']==row['IndustryIndexName'])]['ff_close_sum']
      ff_close = ff_close_list.item() if len(ff_close_list.index) == 1 else np.nan

      ff_os_list = industry_ff_ohlc.loc[(industry_ff_ohlc['IndustryIndexName']==row['IndustryIndexName'])]['os_sum']
      ff_os = ff_os_list.item() if len(ff_os_list.index) == 1 else np.nan

      ff_earnings_list = industry_ff_ohlc.loc[(industry_ff_ohlc['IndustryIndexName']==row['IndustryIndexName'])]['earnings']
      ff_earnings = ff_earnings_list.item() if len(ff_earnings_list.index) == 1 else np.nan

      ff_equity_list = industry_ff_ohlc.loc[(industry_ff_ohlc['IndustryIndexName']==row['IndustryIndexName'])]['sum_equity']
      ff_equity = ff_equity_list.item() if len(ff_equity_list.index) == 1 else np.nan

      prev_earnings_list = industry_ff_ohlc.loc[(industry_ff_ohlc['IndustryIndexName']==row['IndustryIndexName'])]['prev_earnings']
      prev_earnings = prev_earnings_list.item() if len(prev_earnings_list.index) == 1 else np.nan

      prev_equity_list = industry_ff_ohlc.loc[(industry_ff_ohlc['IndustryIndexName']==row['IndustryIndexName'])]['prev_equity']
      prev_equity = prev_equity_list.item() if len(prev_equity_list.index) == 1 else np.nan

      divisor_list = industry_divisor.loc[(industry_divisor['IndexName']==row['IndustryIndexName'])]['Divisor']
      divisor = divisor_list.item() if len(divisor_list.index) == 1 else np.nan

      sum_os_close_list = industry_ff_ohlc.loc[(industry_ff_ohlc['IndustryIndexName']==row['IndustryIndexName'])]['sum_os_close']
      sum_os_close = sum_os_close_list.item() if len(sum_os_close_list.index) == 1 else np.nan

      if np.isnan(divisor) or divisor == 0:
        print("Devisor gen_industry_index: ", divisor)


      div_open = ff_open/divisor if divisor != 0 and not np.isnan(ff_open) else np.nan
      div_high = ff_high/divisor if divisor != 0 and not np.isnan(ff_high) else np.nan
      div_low = ff_low/divisor if divisor != 0 and not np.isnan(ff_low) else np.nan
      div_close = ff_close/divisor if divisor != 0 and not np.isnan(ff_close) else np.nan


      pe = (sum_os_close)/ff_earnings if ff_earnings !=0 else np.nan
      eps = ff_earnings/ff_equity if ff_equity !=0 else np.nan
      eps_back = prev_earnings/prev_equity if prev_equity !=0 else np.nan
      earnings_growth = ((eps-eps_back)/abs(eps_back))*100 if eps_back != 0 else np.nan


      industry_ff_ohlc.loc[index, 'Open'] = div_open
      industry_ff_ohlc.loc[index, 'High'] = div_high
      industry_ff_ohlc.loc[index, 'Low'] = div_low
      industry_ff_ohlc.loc[index, 'Close'] = div_close
      industry_ff_ohlc.loc[index, 'PE'] = pe
      industry_ff_ohlc.loc[index, 'EPS'] = eps
      industry_ff_ohlc.loc[index, 'Earnings Growth'] = earnings_growth


    return industry_ff_ohlc

In [7]:
date = datetime.datetime.strptime('2020-01-01', "%Y-%m-%d").date()

In [ ]:
ind_ind = gen_industry_index(conn,'2020-01-01')

In [ ]:
sub_ind = gen_sub_industry_index(conn, '2020-01-01')

In [ ]:
sub_ind

In [22]:
def calc_rank_index(industry_index, sub_industry_index,conn, date):
    """ calculating the rank for sector, subsector and industry.

    Args:
        sector_index = data of Open, close, high, low, PE , EPS
                      and Earning Growth of sector index.

        subsector_index = data of Open, close, high, low, PE , EPS
                        and Earning Growth of subsector index.

        industry_index = data of Open, close, high, low, PE , EPS
                        and Earning Growth of industry index.

    Operation:
        Fetch the data from IndexHistory and calculate the value for six month change
        for sector index, subsector index and industry index,
        change six month = (curr close - prev close / prev close)* 100
        index = change * 100.

    Return:
        Rank value of Sector Index, SubSector Index and Industry index.
    """

    six_month_back = '01-06-2019'

    indexlist_history_sql = 'SELECT DISTINCT ON("TICKER") * FROM public."IndexHistory" \
                            where "DATE" <= \''+six_month_back+'\' \
                            ORDER BY "TICKER", "DATE" DESC; '
    index_history = sqlio.read_sql_query(indexlist_history_sql, con = conn)


    #Sector rank
    # if not(sector_index.empty):

    #   for index, row in sector_index.iterrows():

    #     prev_close_list = index_history.loc[(index_history['TICKER']==row['SectorIndexName'])]['CLOSE']
    #     prev_close = prev_close_list.item() if len(prev_close_list.index) == 1 else np.nan

    #     current_close_list = sector_index.loc[(sector_index['SectorIndexName']==row['SectorIndexName'])]['Close']
    #     current_close = current_close_list.item() if len(current_close_list.index) == 1 else np.nan

    #     change_six_months = (current_close-prev_close)/prev_close * 100 if prev_close != np.nan else 0

    #     sector_index.loc[index, 'Change'] = change_six_months

    #   sector_index['Rank'] = sector_index['Change'].rank(ascending=True, pct=True) * 100

    #   sector_index = sector_index.rename(columns = {"SectorIndexName":"IndexName", "Sector": "Index"})

    # else:

    #   print("No Sector data")


    # #SubSector Rank
    # if not(subsector_index.empty):

    #   for index, row in subsector_index.iterrows():

    #     prev_close_list = index_history.loc[(index_history['TICKER']==row['SubSectorIndexName'])]['CLOSE']
    #     prev_close = prev_close_list.item() if len(prev_close_list.index) == 1 else np.nan

    #     current_close_list = subsector_index.loc[(subsector_index['SubSectorIndexName']==row['SubSectorIndexName'])]['Close']
    #     current_close = current_close_list.item() if len(current_close_list.index) == 1 else np.nan

    #     change_six_months = (current_close-prev_close)/prev_close * 100 if prev_close != np.nan else 0

    #     subsector_index.loc[index, 'Change'] = change_six_months

    #   subsector_index['Rank'] = subsector_index['Change'].rank(ascending=True, pct=True) * 100

    #   subsector_index = subsector_index.rename(columns = {"SubSectorIndexName":"IndexName", "SubSector": "Index"})

    # else:

    #   print("No Subsector data")


    #Industry Rank
    if not(industry_index.empty):

      for index, row in industry_index.iterrows():

        prev_close_list = index_history.loc[(index_history['TICKER']==row['IndustryIndexName'])]['CLOSE']
        prev_close = prev_close_list.item() if len(prev_close_list.index) == 1 else np.nan

        current_close_list = industry_index.loc[(industry_index['IndustryIndexName']==row['IndustryIndexName'])]['Close']
        current_close = current_close_list.item() if len(current_close_list.index) == 1 else np.nan

        change_six_months = (current_close-prev_close)/prev_close * 100 if prev_close != np.nan else 0

        industry_index.loc[index, 'Change'] = change_six_months

      industry_index['Rank'] = industry_index['Change'].rank(ascending=True, pct=True) * 100

      industry_index = industry_index.rename(columns = {"IndustryIndexName":"IndexName", "Industry": "Index"})

    else:

      print("No Industry data")
      
    #SubIndustry Rank
    if not(sub_industry_index.empty):
    
      for index, row in sub_industry_index.iterrows():

        prev_close_list = index_history.loc[(index_history['TICKER']==row['SubIndustryIndexName'])]['CLOSE']
        prev_close = prev_close_list.item() if len(prev_close_list.index) == 1 else np.nan

        current_close_list = sub_industry_index.loc[(sub_industry_index['SubIndustryIndexName']==row['SubIndustryIndexName'])]['Close']
        current_close = current_close_list.item() if len(current_close_list.index) == 1 else np.nan

        change_six_months = (current_close-prev_close)/prev_close * 100 if prev_close != np.nan else 0

        sub_industry_index.loc[index, 'Change'] = change_six_months

      sub_industry_index['Rank'] = sub_industry_index['Change'].rank(ascending=True, pct=True) * 100

      sub_industry_index = sub_industry_index.rename(columns = {"SubIndustryIndexName":"IndexName", "SubIndustry": "Index"})

    else:

      print("No SubIndustry data")


    irs_list = pd.concat([industry_index, sub_industry_index], axis=0, sort=True)
    print('irslist: ',irs_list)


    return irs_list

In [ ]:
rank = calc_rank_index(ind_ind,sub_ind, conn, '01-01-2020')

In [ ]:
import os
import pandas as pd
from utils.db_helper import DB_Helper
import psycopg2

missing_dir = os.path.join(os.getcwd(), 'missing')

listOfcsv = {}

conn = DB_Helper().db_connect()
cur = conn.cursor()

# recursively iterate over missing
for root, dirs, files in os.walk(missing_dir):
    for file in files:
        if file.endswith(".csv") and not root == 'c:\\BravisaProject\\app\missing\\Bravisa FullData1997 to Sept 2016' and not file.startswith('.') and not root=='c:\\BravisaProject\\app\missing\\FB2904202402':
            listOfcsv[file] = file
print(listOfcsv)
for root, dirs, files in os.walk(missing_dir):
    for file in files:
        if file.endswith(".csv") and not root == 'c:\\BravisaProject\\app\missing\\Bravisa FullData1997 to Sept 2016' and not file.startswith('.') and not root=='c:\\BravisaProject\\app\missing\\FB2904202402' and not file.startswith('Missing'):
            
            if file.endswith(".csv") and file in listOfcsv:
            #print file path
                print(os.path.join(root, file))
                
                # if file == 'ConsolidatedHalfyearlyResults.csv':
                #     print(file)                    
                #     #insert into database
                #     try:
                #         copy_sql = """
                #         COPY "public"."ConsolidatedHalfYearlyResults" FROM stdin WITH CSV HEADER
                #         DELIMITER as ','
                #         """
                #         with open(os.path.join(root,file), 'r') as f:
                #             cur.copy_expert(sql=copy_sql, file=f)
                #             conn.commit()
                #             f.close()    
                #     except:
                #         print("Error in insert_consolidatedhalfyearlyresults()", flush = True)
                #         conn.rollback()
                #         import traceback; traceback.print_exc()
                
                # if file == 'ConsolidatedNinemonthsResults.csv':
                #     print(file)
                #     #insert into database
                #     try:
                #         copy_sql = """
                #         COPY "public"."ConsolidatedNineMonthsResults" FROM stdin WITH CSV HEADER
                #         DELIMITER as ','
                #         """
                #         with open(os.path.join(root,file), 'r') as f:
                #             cur.copy_expert(sql=copy_sql, file=f)
                #             conn.commit()
                #             f.close()    
                #     except:
                #         print("Error in insert_consolidatedninemonthsresults()", flush = True)
                #         conn.rollback()
                #         import traceback; traceback.print_exc()
                        
                # if file == 'HalfyearlyResults.csv':
                #     print(file)
                #     #insert into database
                #     try:
                #         copy_sql = """
                #         COPY "public"."HalfYearlyResults" FROM stdin WITH CSV HEADER
                #         DELIMITER as ','
                #         """
                #         with open(os.path.join(root,file), 'r') as f:
                #             cur.copy_expert(sql=copy_sql, file=f)
                #             conn.commit()
                #             f.close()    
                #     except:
                #         print("Error in insert_halfyearlyresults()", flush = True)
                #         conn.rollback()
                #         import traceback; traceback.print_exc()
                        
                if file == 'NinemonthsResults.csv':
                    print(file)
                    #insert into database
                    try:
                        copy_sql = """
                        COPY "public"."NinemonthsResults" FROM stdin WITH CSV HEADER
                        DELIMITER as ','
                        """
                        with open(os.path.join(root,file), 'r') as f:
                            cur.copy_expert(sql=copy_sql, file=f)
                            conn.commit()
                            f.close()    
                    except:
                        print("Error in insert_ninemonthsresults()", flush = True)
                        conn.rollback()
                        import traceback; traceback.print_exc()
                        
                # if file == 'FinanceBankingVI.csv':
                #     print(file)
                #     #insert into database
                #     try:
                #         copy_sql = """
                #         COPY "public"."FinanceBankingVI" FROM stdin WITH CSV HEADER
                #         DELIMITER as ','
                #         """
                #         with open(os.path.join(root,file), 'r') as f:
                #             cur.copy_expert(sql=copy_sql, file=f)
                #             conn.commit()
                #             f.close()    
                #     except:
                #         print("Error in insert_financebankingvi()", flush = True)
                #         conn.rollback()
                #         import traceback; traceback.print_exc()
                
                # if file == 'FinanceConsolidatedBankingVI.csv':
                #     print(file)
                #     #insert into database
                #     try:
                #         copy_sql = """
                #         COPY "public"."FinanceConsolidatedBankingVI" FROM stdin WITH CSV HEADER
                #         DELIMITER as ','
                #         """
                #         with open(os.path.join(root,file), 'r') as f:
                #             cur.copy_expert(sql=copy_sql, file=f)
                #             conn.commit()
                #             f.close()    
                #     except:
                #         print("Error in insert_financeconsolidatedbankingvi()", flush = True)
                #         conn.rollback()
                #         import traceback; traceback.print_exc()
                        
                # if file == 'FinanceConsolidatedNonBankingVI.csv':
                #     print(file)
                #     #insert into database
                #     try:
                #         copy_sql = """
                #         COPY "public"."FinanceConsolidatedNonBankingVI" FROM stdin WITH CSV HEADER
                #         DELIMITER as ','
                #         """
                #         with open(os.path.join(root,file), 'r') as f:
                #             cur.copy_expert(sql=copy_sql, file=f)
                #             conn.commit()
                #             f.close()    
                #     except:
                #         print("Error in insert_financeconsolidatednonbankingvi()", flush = True)
                #         conn.rollback()
                #         import traceback; traceback.print_exc()
                        
                # if file == 'FinanceNonBankingVI.csv':
                #     print(file)
                #     #insert into database
                #     try:
                #         copy_sql = """
                #         COPY "public"."FinanceNonBankingVI" FROM stdin WITH CSV HEADER
                #         DELIMITER as ','
                #         """
                #         with open(os.path.join(root,file), 'r') as f:
                #             cur.copy_expert(sql=copy_sql, file=f)
                #             conn.commit()
                #             f.close()    
                #     except:
                #         print("Error in insert_financenonbankingvi()", flush = True)
                #         conn.rollback()
                #         import traceback; traceback.print_exc()
                
                # if file == 'RatiosBankingVI.csv':
                #     print(file)
                #     #insert into database
                #     try:
                #         copy_sql = """
                #         COPY "public"."RatiosBankingVI" FROM stdin WITH CSV HEADER
                #         DELIMITER as ','
                #         """
                #         with open(os.path.join(root,file), 'r') as f:
                #             cur.copy_expert(sql=copy_sql, file=f)
                #             conn.commit()
                #             f.close()    
                #     except:
                #         print("Error in insert_ratiosbankingvi()", flush = True)
                #         conn.rollback()
                #         import traceback; traceback.print_exc()
                        
                # if file == 'RatiosConsolidatedBankingVI.csv': 
                #     print(file)
                #     #insert into database
                #     try:
                #         copy_sql = """
                #         COPY "public"."RatiosConsolidatedBankingVI" FROM stdin WITH CSV HEADER
                #         DELIMITER as ','
                #         """
                #         with open(os.path.join(root,file), 'r') as f:
                #             cur.copy_expert(sql=copy_sql, file=f)
                #             conn.commit()
                #             f.close()    
                #     except:
                #         print("Error in insert_ratiosconsolidatedbankingvi()", flush = True)
                #         conn.rollback()
                #         import traceback; traceback.print_exc()
                        
                # if file == 'RatiosConsolidatedNonBankingVI.csv':
                #     print(file)
                #     #insert into database
                #     try:
                #         copy_sql = """
                #         COPY "public"."RatiosConsolidatedNonBankingVI" FROM stdin WITH CSV HEADER
                #         DELIMITER as ','
                #         """
                #         with open(os.path.join(root,file), 'r') as f:
                #             cur.copy_expert(sql=copy_sql, file=f)
                #             conn.commit()
                #             f.close()    
                #     except:
                #         print("Error in insert_ratiosconsolidatednonbankingvi()", flush = True)
                #         conn.rollback()
                #         import traceback; traceback.print_exc()
                        
                # if file == 'RatiosNonBankingVI.csv':
                #     print(file)
                #     #insert into database
                #     try:
                #         copy_sql = """
                #         COPY "public"."RatiosNonBankingVI" FROM stdin WITH CSV HEADER
                #         DELIMITER as ','
                #         """
                #         with open(os.path.join(root,file), 'r') as f:
                #             cur.copy_expert(sql=copy_sql, file=f)
                #             conn.commit()
                #             f.close()    
                #     except:
                #         print("Error in insert_ratiosnonbankingvi()", flush = True)
                #         conn.rollback()
                #         import traceback; traceback.print_exc()
                        
                # if file == 'SchemeNAVDetails.csv':
                #     print(file)
                #     #insert into database
                #     try:
                #         copy_sql = """
                #         COPY "public"."SchemeNAVDetails" FROM stdin WITH CSV HEADER
                #         DELIMITER as ','
                #         """
                #         with open(os.path.join(root,file), 'r') as f:
                #             cur.copy_expert(sql=copy_sql, file=f)
                #             conn.commit()
                #             f.close()    
                #     except:
                #         print("Error in insert_schemenavdetails()", flush = True)
                #         conn.rollback()
                #         import traceback; traceback.print_exc()
                        

In [12]:
import os 
from utils.db_helper import DB_Helper
import psycopg2

missing_dir = os.path.join(os.getcwd(), 'missing')

#insert missingresults
conn = DB_Helper().db_connect()
cur = conn.cursor()

#insert MissingQuarterlyResults
filename = missing_dir + '/MissingQuarterlyResults.csv' 
try:
    copy_sql = """
    COPY "public"."QuarterlyResults" FROM stdin WITH CSV HEADER
    DELIMITER as ','
    """
    with open(filename, 'r') as f:
        cur.copy_expert(sql=copy_sql, file=f)
        conn.commit()
        f.close()
except:
    print("Error in insert_missingquarterlyresults()", flush = True)
    conn.rollback()
    import traceback; traceback.print_exc()

#insert MissingConsolidatedQuarterlyResults
filename = missing_dir + '/MissingConsolidatedQuarterlyResults.csv' 

try:
    copy_sql = """
    COPY "public"."ConsolidatedQuarterlyResults" FROM stdin WITH CSV HEADER
    DELIMITER as ','
    """
    with open(filename
                , 'r') as f:
            cur.copy_expert(sql=copy_sql, file=f)
            conn.commit()
            f.close()       
except:
    print("Error in insert_missingconsolidatedquarterlyresults()", flush = True)
    conn.rollback()
    import traceback; traceback.print_exc()

In [ ]:
# delete scripts for reports
def delete_EPS(conn, cur, start_date, end_date):
	try:

		eps_sql = """DELETE FROM "Reports"."EPS"
					WHERE "EPSDate" <= %s AND "EPSDate" >= %s""" 
		cur.execute(eps_sql, (end_date, start_date))		
		conn.commit()
		print("Deleted EPS data")	
		STANDALONE_EPS_sql = """ DELETE FROM "Reports"."STANDALONE_EPS"
					WHERE "EPSDate" <= %s AND "EPSDate" >= %s"""
		cur.execute(STANDALONE_EPS_sql, (end_date, start_date))
		conn.commit()	
		print("Deleted STANDALONE_EPS data")
		Consolidated_EPS_sql = """ DELETE FROM "Reports"."Consolidated_EPS"
					WHERE "EPSDate" <= %s AND "EPSDate" >= %s"""
		cur.execute(Consolidated_EPS_sql, (end_date, start_date))
		conn.commit()
		print("Deleted Consolidated_EPS data")
		QaurterlyEPS_sql = """ DELETE FROM public."QuarterlyEPS"
					WHERE "YearEnding" >= %s AND "YearEnding" <= %s"""
		cur.execute(QaurterlyEPS_sql, (start_date, end_date))
		conn.commit()
		print("Deleted QuarterlyEPS data")
		ConsolidatedQuarterlyEPS_sql = """ DELETE FROM public."ConsolidatedQuarterlyEPS"
					WHERE "YearEnding" >= %s AND "YearEnding" <= %s"""
		cur.execute(ConsolidatedQuarterlyEPS_sql, (start_date,end_date))
		conn.commit()
		print("Deleted ConsolidatedQuarterlyEPS data")
		TTMsql = """ DELETE FROM public."TTM"
					WHERE "YearEnding" >= %s AND "YearEnding" <= %s"""
		cur.execute(TTMsql, (start_date, end_date))
		conn.commit()	
		print("Deleted TTM data")
		ConsolidatedTTMsql = """ DELETE FROM public."ConsolidatedTTM"
					WHERE "YearEnding" >= %s AND "YearEnding" <= %s"""
		cur.execute(ConsolidatedTTMsql, (start_date, end_date))
		conn.commit()
		print("Deleted ConsolidatedTTM data")	

	except Exception as e:
		print(f"Error deleting EPS data: {e}")
		import traceback; traceback.print_exc()	
  
def delete_smr(conn,cur, start_date, end_date):
    try:
        SMRsql = """ DELETE FROM "Reports"."SMR"
                    WHERE "SMRDate" >= %s AND "SMRDate" <= %s"""
        cur.execute(SMRsql, (start_date, end_date))
        conn.commit()
        print("Deleted SMR data")
        RatiosMergeListsql = """ DELETE FROM public."RatiosMergeList"
                    WHERE "GenDate" >= %s AND "GenDate" <= %s"""
        cur.execute(RatiosMergeListsql, (start_date, end_date))
        conn.commit()
    except Exception as e:
        print(f"Error deleting SMR data: {e}")
        import traceback; traceback.print_exc()
        
def delPRS(start_date, end_date, conn, cur):
    try:
        sql_PRS = """DELETE FROM "Reports"."PRS" 
                    WHERE "Date" >= %s AND "Date" <= %s"""
        cur.execute(sql_PRS, (start_date, end_date))
        conn.commit()
        print("Deleted the records from PRS table for the date range: ", start_date, " to ", end_date)
        
        sql_PE = """DELETE FROM "PE" 
                    WHERE "GenDate" >= %s AND "GenDate" <= %s"""
        cur.execute(sql_PE, (start_date, end_date))
        conn.commit()
        print("Deleted the records from PE table for the date range: ", start_date, " to ", end_date)
        
        sql_NHNL = """DELETE FROM "Reports"."NewHighNewLow" 
                    WHERE "Date" >= %s AND "Date" <= %s"""
        cur.execute(sql_NHNL, (start_date, end_date))
        conn.commit()
        print("Deleted the records from NHNL table for the date range: ", start_date, " to ", end_date)
    except Exception as e: 
        print(f"Error deleting PRS data: {e}")
        import traceback; traceback.print_exc()

from utils.db_helper import DB_Helper
import psycopg2

conn = DB_Helper().db_connect()
cur = conn.cursor()

start_date = '2024-12-20'
end_date = '2024-12-27'

delete_EPS(conn, cur, start_date, end_date)
delete_smr(conn, cur, start_date, end_date)
delPRS(start_date, end_date, conn, cur)


In [22]:
import pandas as pd

xlsx = pd.read_excel('C:\\BravisaBackup\\MF Scheme code working.xlsx', sheet_name='Sheet1')

column_names = ['scheme_code', 'scheme_name', 'scheme_category', 'date', 'btt_scheme_code', 'btt_scheme_category']

xlsx.columns = column_names

xlsx.to_csv('C:\\BravisaBackup\\mf_category_mapping.csv', index=False)

In [ ]:
import os 
import pandas as pd
import psycopg2
#import timedelta
from datetime import timedelta
from lib import  index_ohlc, split_bonus
from Test_reports import ohlc
#ignore warnings
import warnings
from Test_reports.IRSchecker import IRS

warnings.filterwarnings("ignore")

def delIRS(curr_date, conn, cur):
    sql_IndustryList = """delete from "IndustryList" 
                        where "GenDate" = %s"""
    # execute the query with curr_date
    cur.execute(sql_IndustryList, (curr_date,))
    conn.commit()
    print("Deleted the records from IndustryList table for the date: ", curr_date)
    sql_SectorDivisor = """delete from "SectorDivisor"
                            where "Date" = %s"""
    cur.execute(sql_SectorDivisor, (curr_date,))
    conn.commit()
    print("Deleted the records from SectorDivisor table for the date: ", curr_date)
    sql_SubSecotDivisor = """delete from "SubSectorDivisor" 
                            where "Date" = %s"""
    cur.execute(sql_SubSecotDivisor, (curr_date,))
    conn.commit()
    print("Deleted the records from SubSectorDivisor table for the date: ", curr_date)
    sql_IndustryDivisor = """delete from "IndustryDivisor" 
                            where "Date" = %s"""
    cur.execute(sql_IndustryDivisor, (curr_date,))
    conn.commit()
    print("Deleted the records from IndustryDivisor table for the date: ", curr_date)
    sql_SubIndustryDivisor = """delete from "SubIndustryDivisor" 
                                where "Date" = %s"""
    cur.execute(sql_SubIndustryDivisor, (curr_date,))
    conn.commit()
    print("Deleted the records from SubIndustryDivisor table for the date: ", curr_date)
    sql_SectorIndexList = """delete from "SectorIndexList" 
                            where "GenDate" = %s"""
    cur.execute(sql_SectorIndexList, (curr_date,))
    conn.commit()
    print("Deleted the records from SectorIndexList table for the date: ", curr_date)
    sql_SubSectorIndexList = """delete from "SubSectorIndexList" 
                                where "GenDate" = %s"""
    cur.execute(sql_SubSectorIndexList, (curr_date,))
    conn.commit()
    print("Deleted the records from SubSectorIndexList table for the date: ", curr_date)
    sql_IndustryIndexList = """delete from "IndustryIndexList" 
                                where "GenDate" = %s"""
    cur.execute(sql_IndustryIndexList, (curr_date,))
    conn.commit()
    print("Deleted the records from IndustryIndexList table for the date: ", curr_date)
    sql_SubIndustryIndexList = """delete from "SubIndustryIndexList" 
                                    where "GenDate" = %s""" 
    cur.execute(sql_SubIndustryIndexList, (curr_date,))
    conn.commit()
    print("Deleted the records from SubIndustryIndexList table for the date: ", curr_date)
    sql_IRS = """delete from "Reports"."IRS" 
                where "GenDate" = %s"""
    cur.execute(sql_IRS, (curr_date,))
    conn.commit()
    print("Deleted the records from IRS table for the date: ", curr_date)
    sql_IndexHistory = """delete from "IndexHistory"  
                        where "DATE" = %s """
    cur.execute(sql_IndexHistory, (curr_date,))
    conn.commit() 
    
def insert_shareholding(conn, cur,fbname ):
    """ Insert the ShareHolding data into database

    Operation:
        Set the path for csv file, and fetch the data from ShareHolding.csv file.
        delete the data based on key column CompanyCode and SHPDate,  
        Export data into 'ShareHoldingExport.csv' file and insert into ShareHolding Table.
    """
    fb_name = fbname
    file_path = os.path.join(os.getcwd(), "FBFiles")

    fb_csv_path = os.path.join(file_path, fb_name) 
    fb_csv_file = os.path.join(fb_csv_path ,'ShareHolding.csv')
    
    file_to_check = file_path +'\\'+ fb_name + '\\' + 'ShareHolding.csv'
    
    # print(file_to_check)
    
    if(fbname == 'intermediate_insert'):
        
        file_to_check = 'C:\\Users\\dsram\\BravisaLocalDeploy\\BravisaFiles\\MissingData'+ '\\' + 'ShareHolding.csv'
        fb_csv_file = file_to_check

    if os.path.isfile(file_to_check):
        try:
            table = pd.read_csv(fb_csv_file, engine='python')

            table['SHPDate'] = pd.to_datetime(table['SHPDate'], errors = 'ignore')
            table['SHPDate'] = table['SHPDate'].apply(lambda dt: dt.strftime('%Y-%m-%d')if not pd.isnull(dt) else '')

            table['ModifiedDate'] = pd.to_datetime(table['ModifiedDate'], errors = 'ignore')
            table['ModifiedDate'] = table['ModifiedDate'].apply(lambda dt: dt.strftime('%Y-%m-%d')if not pd.isnull(dt) else '')
            
            #Update Logic - Deletes based on key columns 
            print("Executing delete logic", flush = True)
            table_to_delete = table.groupby(['CompanyCode', 'SHPDate'], as_index=False).count()
            '''self.logfile.write("\tIn function: " + str(fb_csv_file.split("\\")[-1].split(".")[0]) + 
                        "\t\tTotal Loops:" + str(int(table_to_delete.shape[0])) + "\t\tFor file: " + str(fbname) + "\n")
            self.logfile.flush()'''
            for index,row in table_to_delete.iterrows():
                # self.logfile.write("\tIn function: " + str(fb_csv_file.split("\\")[-1].split(".")[0]) + 
                # 		"\t\tTotal Loops:" + str(int(table_to_delete.shape[0])) + "\t\tFor file: " + str(fbname) + "\n")
                # self.logfile.flush()
                sql = 'DELETE FROM public."ShareHolding" WHERE "CompanyCode" =\'' +str(row['CompanyCode']) + '\' AND  "SHPDate"=\'' + str(row['SHPDate']) + '\';'
                ##print(sql, flush = True)
                cur.execute(sql)
                conn.commit()
            

            print("Inserting Data into ShareHolding", flush = True)
            exportfilename = "ShareHoldingExport.csv"
            exportfile = open(exportfilename,"w",encoding='utf-8')
            table.to_csv(exportfile, header=True, index=False, lineterminator='\r')
            exportfile.close()
            
            copy_sql = """
                COPY "public"."ShareHolding" FROM stdin WITH CSV HEADER
                DELIMITER as ','
                """
            with open(exportfilename, 'r') as f:
                cur.copy_expert(sql=copy_sql, file=f)
                conn.commit()
                f.close()
            os.remove(exportfilename)
        except:
            print("Error in insert_shareholding()", flush = True)
            conn.rollback()

    else:
        print("File not found: " + fb_csv_file, flush = True)

def delete_OHLC(conn, cur, curr_date, end_date):
    try:
        ohlc_sql = f"""DELETE FROM public."OHLC" WHERE "Date" >= '{curr_date}' AND "Date" <= '{end_date}'"""
        cur.execute(ohlc_sql)
        conn.commit()
        print("Deleted OHLC data")
    except Exception as e:
        print(f"Error deleting OHLC data: {e}")

    try:
        nse_sql = f"""DELETE FROM public."NSE" WHERE "TIMESTAMP" >= '{curr_date}' AND "TIMESTAMP" <= '{end_date}'"""
        cur.execute(nse_sql)
        conn.commit()
        print("Deleted NSE data")
    except Exception as e:
        print(f"Error deleting NSE data: {e}")

    try:
        bse_sql = f"""DELETE FROM public."BSE" WHERE "TRADING_DATE" >= '{curr_date}' AND "TRADING_DATE" <= '{end_date}'"""
        cur.execute(bse_sql)
        conn.commit()
        print("Deleted BSE data")
    except Exception as e:
        print(f"Error deleting BSE data: {e}")

# Establish database connection
try:
    conn = psycopg2.connect(
        host="localhost",
        database="BravisaDB",
        user="postgres",
        password="edsols"
    )
    cur = conn.cursor()
    date_range = pd.date_range(start='2020-08-01',end='2025-01-03')
    start_date = date_range[0]
    end_date = date_range[-1]
    delete_OHLC(conn, cur, start_date, end_date)
    date_format = "%d%m%Y"
    current_date = start_date
    while current_date <= end_date:
        if current_date.weekday() == 0:  # Monday
            prev_date = current_date - timedelta(days=2)  # Saturday
        else:
            prev_date = current_date - timedelta(days=1)  # Previous day

        print(current_date)
        insert_shareholding(conn, cur, "FB" + prev_date.strftime(date_format) + "03")
        insert_shareholding(conn, cur, "FB" + current_date.strftime(date_format) + "01")
        insert_shareholding(conn, cur, "FB" + current_date.strftime(date_format) + "02")
        print("Inserted: shareholding for : ", current_date)
        try:
            ohlc.main(current_date)
            # ohlc_nse, ohlc_bse, ohlc_joined, ohlc_bg,  ohlc_full = ohlc.main(current_date)
            split_bonus.main(current_date)
            delIRS(current_date, conn, cur)    
            btt_list, ohlc_irs, btt_ohlc_merge, ohlc_change, ohlc_backgroundinfo_merge, industry_merge, industry_list_calc_free_float, \
            industry_list_ff_ohlc, master_list, sector_divisor, subsector_divisor, industry_divisor, sub_industry_divisor, sector_divisor_index,\
            subsector_divisor_index, industry_divisor_index, sub_industry_divisor_index, sector_index, subsector_index, industry_index,\
            sub_industry_index,irs_list_rank, irs_list, irs_pe, industry_list = IRS().gen_irs_current(current_date,conn, cur)
            
            
        except FileNotFoundError:
            print(" there is no file for the date: ", current_date)    
        current_date += timedelta(days=1)
except Exception as e:
    print(f"Error connecting to the database: {e}")
    import traceback; traceback.print_exc();
finally:
    if cur:
        cur.close()
    if conn:
        conn.close()



In [ ]:
# SELECT distinct "SecurityCode" FROM public."SchemeNAVCurrentPrices"

import pandas as pd
import psycopg2
from utils.db_helper import DB_Helper

conn = DB_Helper().db_connect()
cur = conn.cursor()
min_len = 100

sql = """SELECT distinct "SecurityCode" FROM public."SchemeNAVCurrentPrices" """
df = pd.read_sql_query(sql, conn)

print(len(df))

df['SecurityCode'] = df['SecurityCode'].astype(str)

for index, row in df.iterrows():
        print(row['SecurityCode'])
        print(len(row['SecurityCode']))
        print(min_len)
        if len(row['SecurityCode']) < min_len:
            min_len = len(row['SecurityCode'])

print(min_len)

In [1]:
import datetime
import pandas as pd
# from utils.report_delete import deleteFRS
from utils.db_helper import DB_Helper    

conn= DB_Helper().db_connect()
cur = conn.cursor()

start_date = '2025-01-03'
end_date = '2025-01-03'

def deleteFRS(curr_date, end_date):

    conn = DB_Helper().db_connect()
    cur = conn.cursor()

    print("Deleting MFMergeList data for:",curr_date)
    mfmerge_sql = 'DELETE FROM public."MFMergeList" WHERE "GenDate">=\'' + (curr_date) + '\' AND "GenDate" <=\'' + (end_date) + '\';'
    cur.execute(mfmerge_sql)
    conn.commit()

    print("Deleting FRS MF Rank data for:",curr_date)
    frs_navrank_sql = 'DELETE FROM "Reports"."FRS-MFRank" WHERE "Date">=\'' + (curr_date) + '\' AND "Date" <= \'' + (end_date) + '\';'
    cur.execute(frs_navrank_sql)
    conn.commit()
    
    print("Deleting FRS-NAVRank data for:",curr_date)
    frs_nav_sql = 'DELETE FROM "Reports"."FRS-NAVRank" WHERE "Date">=\'' + (curr_date) + '\' AND "Date" <= \'' + (end_date) + '\';'
    cur.execute(frs_nav_sql)
    conn.commit()

    print("Deleting FRS-NAV Category Avg data for:",curr_date)
    frs_sql = 'DELETE FROM "Reports"."FRS-NAVCategoryAvg" WHERE "Date">=\'' + (curr_date) + '\' AND "Date" <= \'' + (end_date) + '\';'         
    cur.execute(frs_sql)
    conn.commit()

    print("deleting MF ohlc for:", curr_date)
    mfohlc_sql = 'delete from public.mf_ohlc where date >= \'' + (curr_date) + '\' AND date <= \'' + (end_date) + '\';'         
    cur.execute(mfohlc_sql)
    conn.commit()

    conn.close()
    
deleteFRS(start_date, end_date)

Deleting MFMergeList data for: 2025-01-03
Deleting FRS MF Rank data for: 2025-01-03
Deleting FRS-NAVRank data for: 2025-01-03
Deleting FRS-NAV Category Avg data for: 2025-01-03
deleting MF ohlc for: 2025-01-03


In [2]:
from utils.db_helper import DB_Helper   
from reports.FRS import FRS
import pandas as pd

conn = DB_Helper().db_connect()
cur = conn.cursor()

start_date = '2025-01-03'
end_date = '2025-01-03'

for curr_date in pd.date_range(start=start_date, end=end_date):
        FRS().generate_current_mfrank(curr_date,conn,cur)
        # scheme_master_nav, merge_scheme_nav, mf_category_mapping, scheme_nav_current_prices, scheme_rank, scheme_nav_category_avg = FRS().calc_nav_rank(conn,cur, curr_date)
        FRS().generate_current_nav_rank(curr_date,conn,cur)


Generating MF List for today: 2025-01-03 00:00:00
Filtering schemes from Scheme Master


c:\BravisaProject\app\reports\FRS.py:57: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  scheme_master_list = sqlio.read_sql_query(scheme_master_sql, con = conn)
c:\BravisaProject\app\reports\FRS.py:67: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  scheme_aum_list = sqlio.read_sql_query(scheme_aum_sql, con = conn)


Length of scheme master list: 1380
Merging filtered list with schemewise portfolio


c:\BravisaProject\app\reports\FRS.py:109: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  scheme_wise_portfolio_list = sqlio.read_sql_query(scheme_wise_portfolio_sql, con = conn)


Merging with Indutry mapping
Calculating Market Cap


...

Index Size for MF:  1101
Inserting into table
Completed MF Rank generation

Generating NAV Rank and category average for today: 2025-01-03 00:00:00
Filtering schemes from scheme master


c:\BravisaProject\app\reports\FRS.py:345: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  scheme_list = sqlio.read_sql_query(scheme_sql, con = conn)
c:\BravisaProject\app\reports\FRS.py:353: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  scheme_aum_list = sqlio.read_sql_query(scheme_aum_sql, con = conn)


Merging with SchemeNavMaster
Getting BTT MF Categories
Getting Scheme NAV current prices data


c:\BravisaProject\app\reports\FRS.py:383: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  scheme_nav_master_list = sqlio.read_sql_query(scheme_nav_master_sql, con = conn)
c:\BravisaProject\app\reports\FRS.py:414: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  btt_mf_category = sqlio.read_sql_query(sql, con = conn)
c:\BravisaProject\app\reports\FRS.py:448: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  scheme_nav_prices_list = sqlio.read_sql_query(scheme_nav_sql, con = conn)


Calculating Scheme Rank
Inserting Scheme NAV Rank in table
Inserting Scheme NAV List: 
Calculating average of each group
Inserting NAV Category Average in table
Inserting Scheme NAV category average List: 
Completed NAV Rank and category average


In [4]:
import pandas as pd
from decimal import Decimal, ROUND_DOWN
import numpy as np

# Example DataFrame
data = {'SecurityCode': [14051447.002066001, 12345678.987654321, 98765432.123456789]}
scheme_nav_merge_list = pd.DataFrame(data)

# Truncate values to 6 decimal places without rounding
scheme_nav_merge_list['SecurityCode'] = scheme_nav_merge_list['SecurityCode'].apply(
    lambda x: Decimal(str(x)).quantize(Decimal('1.000000'), rounding=ROUND_DOWN)
)
scheme_nav_merge_list['SecurityCode'] = scheme_nav_merge_list['SecurityCode'].astype(np.float64)

print(scheme_nav_merge_list)
print(scheme_nav_merge_list.dtypes)


   SecurityCode
0  1.405145e+07
1  1.234568e+07
2  9.876543e+07
SecurityCode    float64
dtype: object


In [ ]:
# inserting missing results till 2018 december
from utils.db_helper import DB_Helper
import pandas as pd

db = DB_Helper()
conn = db.db_connect()
cur = conn.cursor()

missing_dir = os.path.join(os.getcwd(), 'missing')

date = datetime.strptime('2018-12-01', '%Y-%m-%d')

MissingQuarterlyResults = pd.read_csv(os.path.join(missing_dir, 'MissingQuarterlyResults.csv'))
MissingQuarterlyResults['ModifiedDate'] = pd.to_datetime(MissingQuarterlyResults['ModifiedDate']) 
# subset of MissingQuarterlyResulst but ModifiedDate less than 2018-12-01
MissingQuarterlyResults = MissingQuarterlyResults[MissingQuarterlyResults['ModifiedDate'].dt.date <= date.date()]
MissingConsolidatedQuarterlyResults = pd.read_csv(os.path.join(missing_dir, 'MissingConsolidatedQuarterlyResults.csv'))
MissingConsolidatedQuarterlyResults['ModifiedDate'] = pd.to_datetime(MissingConsolidatedQuarterlyResults['ModifiedDate'])
MissingConsolidatedQuarterlyResults = MissingConsolidatedQuarterlyResults[MissingConsolidatedQuarterlyResults['ModifiedDate'].dt.date <= date.date()]

print(MissingQuarterlyResults.shape)
print(MissingConsolidatedQuarterlyResults.shape)

# fetch QuarterlyResults till 2018 december
fetch_quarterlyresults_sql = """SELECT * FROM public."QuarterlyResults" WHERE "ModifiedDate" <= '2018-12-01'"""
fetch_consolidatedquarterlyresults_sql = """SELECT * FROM public."ConsolidatedQuarterlyResults" WHERE "ModifiedDate" <= '2018-12-01'"""

quarterlyresults = pd.read_sql_query(fetch_quarterlyresults_sql, conn)
consolidatedquarterlyresults = pd.read_sql_query(fetch_consolidatedquarterlyresults_sql, conn)
print("from db")    
print(quarterlyresults.shape)
print(consolidatedquarterlyresults.shape)

# concatenate the missing results with the fetched results
quarterlyresults = pd.concat([quarterlyresults, MissingQuarterlyResults])
consolidatedquarterlyresults = pd.concat([consolidatedquarterlyresults, MissingConsolidatedQuarterlyResults])
print("after concatenation")
print(quarterlyresults.shape)
print(consolidatedquarterlyresults.shape)

#sort by modified date
quarterlyresults = quarterlyresults.sort_values(by='ModifiedDate')
consolidatedquarterlyresults = consolidatedquarterlyresults.sort_values(by='ModifiedDate')
#drop duplicates
quarterlyresults = quarterlyresults.drop_duplicates(['CompanyCode', 'YearEnding'], keep='last')
consolidatedquarterlyresults = consolidatedquarterlyresults.drop_duplicates(['CompanyCode', 'YearEnding'], keep='last')
print("after dropping duplicates")
print(quarterlyresults.shape)
print(consolidatedquarterlyresults.shape)

# delete the data from the tables
delete_quarterlyresults_sql = """DELETE FROM public."QuarterlyResults" WHERE "ModifiedDate" <= '2018-12-01'"""
delete_consolidatedquarterlyresults_sql = """DELETE FROM public."ConsolidatedQuarterlyResults" WHERE "ModifiedDate" <= '2018-12-01'"""
try:
    cur.execute(delete_quarterlyresults_sql)
    cur.execute(delete_consolidatedquarterlyresults_sql)
    conn.commit()
    print("Deleted data till 2018-12-01")
except Exception as e:
    print(f"Error deleting data: {e}")
    import traceback; traceback.print_exc()
    conn.rollback()



In [ ]:
# inserting missing results till 2018 december
from utils.db_helper import DB_Helper
import pandas as pd

db = DB_Helper()
conn = db.db_connect()
cur = conn.cursor()

missing_dir = os.path.join(os.getcwd(), 'missing')

date = datetime.strptime('2018-12-01', '%Y-%m-%d')

MissingQuarterlyResults = pd.read_csv(os.path.join(missing_dir, 'MissingQuarterlyResults.csv'))
MissingQuarterlyResults['ModifiedDate'] = pd.to_datetime(MissingQuarterlyResults['ModifiedDate']) 
# subset of MissingQuarterlyResulst but ModifiedDate less than 2018-12-01
MissingQuarterlyResults = MissingQuarterlyResults[MissingQuarterlyResults['ModifiedDate'].dt.date <= date.date()]
MissingConsolidatedQuarterlyResults = pd.read_csv(os.path.join(missing_dir, 'MissingConsolidatedQuarterlyResults.csv'))
MissingConsolidatedQuarterlyResults['ModifiedDate'] = pd.to_datetime(MissingConsolidatedQuarterlyResults['ModifiedDate'])
MissingConsolidatedQuarterlyResults = MissingConsolidatedQuarterlyResults[MissingConsolidatedQuarterlyResults['ModifiedDate'].dt.date <= date.date()]

print(MissingQuarterlyResults.shape)
print(MissingConsolidatedQuarterlyResults.shape)

# fetch QuarterlyResults till 2018 december
fetch_quarterlyresults_sql = """SELECT * FROM public."QuarterlyResults" WHERE "ModifiedDate" <= '2018-12-01'"""
fetch_consolidatedquarterlyresults_sql = """SELECT * FROM public."ConsolidatedQuarterlyResults" WHERE "ModifiedDate" <= '2018-12-01'"""

quarterlyresults = pd.read_sql_query(fetch_quarterlyresults_sql, conn)
consolidatedquarterlyresults = pd.read_sql_query(fetch_consolidatedquarterlyresults_sql, conn)
print("from db")    
print(quarterlyresults.shape)
print(consolidatedquarterlyresults.shape)

# concatenate the missing results with the fetched results
quarterlyresults = pd.concat([quarterlyresults, MissingQuarterlyResults])
consolidatedquarterlyresults = pd.concat([consolidatedquarterlyresults, MissingConsolidatedQuarterlyResults])
print("after concatenation")
print(quarterlyresults.shape)
print(consolidatedquarterlyresults.shape)

#sort by modified date
quarterlyresults = quarterlyresults.sort_values(by='ModifiedDate')
consolidatedquarterlyresults = consolidatedquarterlyresults.sort_values(by='ModifiedDate')
#drop duplicates
quarterlyresults = quarterlyresults.drop_duplicates(['CompanyCode', 'YearEnding'], keep='last')
consolidatedquarterlyresults = consolidatedquarterlyresults.drop_duplicates(['CompanyCode', 'YearEnding'], keep='last')
print("after dropping duplicates")
print(quarterlyresults.shape)
print(consolidatedquarterlyresults.shape)

# delete the data from the tables
delete_quarterlyresults_sql = """DELETE FROM public."QuarterlyResults" WHERE "ModifiedDate" <= '2018-12-01'"""
delete_consolidatedquarterlyresults_sql = """DELETE FROM public."ConsolidatedQuarterlyResults" WHERE "ModifiedDate" <= '2018-12-01'"""
try:
    cur.execute(delete_quarterlyresults_sql)
    cur.execute(delete_consolidatedquarterlyresults_sql)
    conn.commit()
    print("Deleted data till 2018-12-01")
except Exception as e:
    print(f"Error deleting data: {e}")
    import traceback; traceback.print_exc()
    conn.rollback()



In [5]:
from utils.db_helper import DB_Helper
import pandas as pd

conn = DB_Helper().db_connect() 
cur = conn.cursor()

# get all the schema names without system schemas
schema_sql = """SELECT schema_name FROM information_schema.schemata WHERE schema_name NOT LIKE 'pg_%' AND schema_name != 'information_schema'"""
schema_list = pd.read_sql_query(schema_sql, conn)
print(schema_list)

# get all the table names from a variable schema
schema_name = 'logs'
tables_sql = f"""SELECT table_name FROM information_schema.tables WHERE table_schema = '{schema_name}'"""
table_list = pd.read_sql_query(tables_sql, conn)
print(table_list)



    schema_name
0        public
1   mf_analysis
2          logs
3  dash_process
4       Reports
          table_name
0               OHLC
1        split_bonus
2  report_generation
3             insert


C:\Users\Admin\AppData\Local\Temp\ipykernel_21540\9311751.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  schema_list = pd.read_sql_query(schema_sql, conn)
C:\Users\Admin\AppData\Local\Temp\ipykernel_21540\9311751.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  table_list = pd.read_sql_query(tables_sql, conn)


In [9]:
from utils.db_helper import DB_Helper
import pandas as pd

db = DB_Helper()

conn = db.db_connect()

date = '2024-09-12'

# Fetch data from PRS
prs_sql = f"""SELECT "NSECode", "Close", "Change52WPercentage", "RS 52W", "Market Cap Value" 
    FROM "Reports"."PRS" WHERE "Date" = '{date}'"""
prs_data = pd.read_sql_query(prs_sql, conn)


# Fetch ranking from EPS
eps_sql = f"""SELECT "NSECode", "Ranking" FROM "Reports"."EPS" WHERE "EPSDate" = '{date}'"""
eps_data = pd.read_sql_query(eps_sql, conn)

# Fetch ranking from EERS
eers_sql = f"""SELECT "NSECode", "Ranking" FROM "Reports"."EERS" WHERE "EERSDate" = '{date}'"""
eers_data = pd.read_sql_query(eers_sql, conn)

# Fetch industry from SMR
smr_sql = f"""SELECT "NSESymbol", "Industry" FROM "Reports"."SMR" WHERE "SMRDate" = '{date}'"""
smr_data = pd.read_sql_query(smr_sql, conn)

# Fetch rank from IRS
irs_sql = f"""SELECT "Index", "Rank" FROM "Reports"."IRS" WHERE "GenDate" = '{date}'"""
irs_data = pd.read_sql_query(irs_sql, conn)

# Merge DataFrames
summary_df = pd.merge(prs_data, eps_data, on='NSECode', how='inner')
# drop rows with missing values in NSECode 
summary_df = summary_df.dropna(subset=['NSECode'])

summary_df = summary_df.merge(eers_data, on="NSECode", how="inner", suffixes=("", "_EERS"))

summary_df = summary_df.merge(smr_data, left_on="NSECode", right_on="NSESymbol", how="inner")

summary_df = summary_df.merge(irs_data, left_on="Industry", right_on="Index", how="inner")


# # Convert numerical values to two decimal places
# summary_df["Market Cap Value"] = summary_df["Market Cap Value"] / 10000000
# summary_df = summary_df.round(2)

# # Selecting necessary columns
# summary_df = summary_df[["NSECode", "Close", "Change52WPercentage", "RS 52W", 
#                 "Ranking", "Ranking_EERS", "Market Cap Value", "Industry", "Rank"]]

# # Renaming columns
# summary_df.columns = ["NSECode", "Close", "Change 52", "RS 52W", 
#             "Ranking (EPS)", "Ranking (EERS)", "Market Cap", "Industry", "Rank (IRS)"]

C:\Users\Admin\AppData\Local\Temp\ipykernel_1764\2576054327.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  prs_data = pd.read_sql_query(prs_sql, conn)
C:\Users\Admin\AppData\Local\Temp\ipykernel_1764\2576054327.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  eps_data = pd.read_sql_query(eps_sql, conn)
C:\Users\Admin\AppData\Local\Temp\ipykernel_1764\2576054327.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  eers_data = pd.read_sql_query(eers_sql, conn)
C:\Users\Admin\AppData\Local\Temp\ipykernel_1764\2576054

In [17]:
from utils.db_helper import DB_Helper   
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

conn = DB_Helper().db_connect()
cur = conn.cursor()

# get all the schema names without system schemas
schema_sql = """SELECT schema_name FROM information_schema.schemata WHERE schema_name NOT LIKE 'pg_%' AND schema_name != 'information_schema'"""
schema_list = pd.read_sql_query(schema_sql, conn)


for scheme in schema_list['schema_name']:
    # get all the table names from a variable schema
    tables_sql = f"""SELECT table_name FROM information_schema.tables WHERE table_schema = '{scheme}'"""
    table_list = pd.read_sql_query(tables_sql, conn)

    for table in table_list['table_name']:
        # get all the column names from a variable table
        columns_sql = f"""SELECT column_name FROM information_schema.columns WHERE table_schema = '{scheme}' AND table_name = '{table}'"""
        columns_list = pd.read_sql_query(columns_sql, conn)

        print(f"Schema: {scheme}, Table: {table} - Columns: {columns_list}")



Schema: public, Table: AnnualGeneralMeeting - Columns:              column_name
0            CompanyCode
1     DateOfAnnouncement
2                AGMDate
3             YearEnding
4                Purpose
5             RecordDate
6   BookClosureStartDate
7     BookClosureEndDate
8                Remarks
9             DeleteFlag
10          ModifiedDate
Schema: public, Table: Auditors - Columns:     column_name
0   Companycode
1    AgencyName
2  ModifiedDate
Schema: public, Table: BSE - Columns:      column_name
0        SC_CODE
1        SC_NAME
2       SC_GROUP
3        SC_TYPE
4           OPEN
5           HIGH
6            LOW
7          CLOSE
8           LAST
9      PREVCLOSE
10     NO_TRADES
11    NO_OF_SHRS
12    NET_TURNOV
13     TDCLOINDI
14     ISIN_CODE
15  TRADING_DATE
Schema: public, Table: BTTDivisor - Columns:      column_name
0      IndexName
1  BTTIndexValue
2        Divisor
3           Date
4             OS
Schema: public, Table: BTTDivisor_DUPLICATED - Columns:      col

## Schema-table-date_column table creation process 

In [ ]:
from utils.db_helper import DB_Helper
import pandas as pd
import datetime
import warnings
warnings.filterwarnings("ignore")

# Connect to the database
conn = DB_Helper().db_connect()
cur = conn.cursor()

# Create the new table
create_table_sql = """
CREATE TABLE IF NOT EXISTS schema_table_date_columns (
    schema_name VARCHAR(255),
    table_name VARCHAR(255),
    date_column VARCHAR(255), 
    insert_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);
"""
cur.execute(create_table_sql)
conn.commit()

# Define the date columns for each table with schema names
date_columns = {
    ('public', 'AnnualGeneralMeeting'): 'ModifiedDate',
    ('public', 'Auditors'): 'ModifiedDate',
    ('public', 'BSE'): 'TRADING_DATE',
    ('public', 'BTTDivisor'): 'Date',
    ('public', 'BTTDivisor_DUPLICATED'): 'Date',
    ('public', 'BTTList'): 'BTTDate',
    ('public', 'BTTListCopy'): 'BTTDate',
    ('public', 'BTTList_DUPLICATED'): 'BTTDate',
    ('public', 'BTTList_STABLE'): 'BTTDate',
    ('public', 'BackgroundInfo'): 'ModifiedDate',
    ('public', 'Bankers'): 'ModifiedDate',
    ('public', 'Blockdeals'): 'DealDate',           # verify
    ('public', 'BoardMeetings'): 'BoardMeetDate',   # verify
    ('public', 'Bonus'): 'DateOfAnnouncement',      # verify
    ('public', 'BulkDeals'): 'DealDate',            
    ('public', 'CapitalStructure'): 'ModifiedDate',
    ('public', 'CompanyMaster'): 'ModifiedDate',
    ('public', 'CompanyNameChange'): 'EffectiveDate',
    ('public', 'ConsolidatedEERSTTM'): 'YearEnding',
    ('public', 'ConsolidatedERSTTM'): 'YearEnding',
    ('public', 'ConsolidatedHalfYearlyResults'): 'YearEnding',
    ('public', 'ConsolidatedNineMonthsResults'): 'YearEnding',
    ('public', 'ConsolidatedQuarterlyEERS'): 'YearEnding',
    ('public', 'ConsolidatedQuarterlyEPS'): 'YearEnding',
    ('public', 'ConsolidatedQuarterlyEPS_DUPLICATED'): 'YearEnding',
    ('public', 'ConsolidatedQuarterlyERS'): 'YearEnding',
    ('public', 'ConsolidatedQuarterlyResults'): 'YearEnding',
    ('public', 'ConsolidatedQuarterlyResults_DUPLICATED'): 'YearEnding',
    ('public', 'ConsolidatedQuarterlyResults_OLD'): 'YearEnding',
    ('public', 'ConsolidatedTTM'): 'YearEnding',
    ('public', 'ConsolidatedTTM_DUPLICATED'): 'YearEnding',
    ('public', 'CurrentData'): 'Yearending',
    ('public', 'Dividend'): 'DateOfAnnouncement',
    ('public', 'EERS_TTM'): 'YearEnding',
    ('public', 'ERS_TTM'): 'YearEnding',
    ('public', 'ErrorLog'): 'TIMESTAMP',
    ('public', 'FinanceBankingVI'): 'YearEnding',
    ('public', 'FinanceConsolidatedBankingVI'): 'YearEnding',
    ('public', 'FinanceConsolidatedNonBankingVI'): 'YearEnding',
    ('public', 'FinanceNonBankingVI'): 'YearEnding',
    ('public', 'FundMaster'): 'ModifiedDate',
    ('public', 'HalfYearlyResults'): 'YearEnding',
    ('public', 'IndexHistory'): 'DATE',
    ('public', 'IndexHistory_DUPLICATED'): 'DATE',
    ('public', 'IndexOHLC'): 'Date',
    ('public', 'IndexOHLC_DUPLICATED'): 'Date',
    ('public', 'IndividualHolding'): 'ShareHoldingDate',
    ('public', 'IndustryDivisor'): 'Date',
    ('public', 'IndustryDivisor_DUPLICATED'): 'Date',
    ('public', 'IndustryIndexList'): 'GenDate',
    ('public', 'IndustryIndexList_DUPLICATED'): 'GenDate',
    ('public', 'IndustryList'): 'GenDate',
    ('public', 'IndustryList_DUPLICATED'): 'GenDate',
    ('public', 'IndustryMapping'): 'ModifiedDate',
    ('public', 'IndustryMaster'): 'ModifiedDate',
    ('public', 'KeyExecutives'): 'ModifiedDate',
    ('public', 'Locations'): 'ModifiedDate',
    ('public', 'MFInvestments'): 'MFDate',
    ('public', 'MFMergeList'): 'HoldingDate',
    ('public', 'ManagementTeam'): 'ModifiedDate',
    ('public', 'NSE'): 'TIMESTAMP',
    ('public', 'News'): 'NewsDate',
    ('public', 'NinemonthsResults'): 'YearEnding',
    ('public', 'OHLC'): 'Date',
    ('public', 'OHLC_DUPLICATED'): 'Date',
    ('public', 'PE'): 'GenDate',
    ('public', 'PE_DUPLICATED'): 'GenDate',
    ('public', 'PledgeShares'): 'SHPDate',
    ('public', 'Products'): 'YearEnding',
    ('public', 'QuarterlyEERS'): 'YearEnding',
    ('public', 'QuarterlyEPS'): 'YearEnding',
    ('public', 'QuarterlyEPS_DUPLICATED'): 'YearEnding',
    ('public', 'QuarterlyEPS_ToBeDeleted'): 'YearEnding',
    ('public', 'QuarterlyERS'): 'YearEnding',
    ('public', 'QuarterlyResults'): 'YearEnding',
    ('public', 'QuarterlyResults_DUPLICATED'): 'YearEnding',
    ('public', 'QuarterlyResults_TEMP'): 'YearEnding',
    ('public', 'RatiosBankingVI'): 'YearEnding',
    ('public', 'RatiosConslidatedNonBankingVI'): 'YearEnding',
    ('public', 'RatiosConsolidatedBankingVI'): 'YearEnding',
    ('public', 'RatiosMergeList'): 'GenDate',
    ('public', 'RatiosMergeList_DUPLICATED'): 'GenDate',
    ('public', 'RatiosNonBankingVI'): 'YearEnding',
    ('public', 'RawMaterials'): 'YearEnding',
    ('public', 'Registrars'): 'ModifiedDate',
    ('public', 'Rights'): 'DateOfAnnouncement',
    ('public', 'SchemeBoardOfAMC'): 'ModifyDate',
    ('public', 'SchemeBoardOfTrustees'): 'ModifiedDate',
    ('public', 'SchemeBonusDetails'): 'BonusDate',
    ('public', 'SchemeCategoryDetails'): 'CategoryNavDate',
    ('public', 'SchemeCorporateDividendDetails'): 'DividendDate',
    ('public', 'SchemeDividendDetails'): 'DividendDate',
    ('public', 'SchemeFundManagerProfile'): 'ModifiedDate',
    ('public', 'SchemeLocations'): 'ModifiedDate',
    ('public', 'SchemeMaster'): 'ModifiedDate',
    ('public', 'SchemeMasterCopy'): 'ModifiedDate',
    ('public', 'SchemeMaster_Join_DUPLICATED'): 'ModifiedDate',
    ('public', 'SchemeMaster_OLD'): 'ModifiedDate',
    ('public', 'SchemeNAVCurrentPrices'): 'DateTime',
    ('public', 'SchemeNAVDetails'): 'NAVDate',
    ('public', 'SchemeNAVMaster'): 'ModifiedDate',
    ('public', 'SchemeObjectives'): 'ModifiedDate',
    ('public', 'SchemePortfolioHeader'): 'HoldingDate',
    ('public', 'SchemeRegistrars'): 'ModifiedDate',
    ('public', 'SchemeSplitsDetails'): 'SplitsDate',
    ('public', 'SchemeTaxBenefit'): 'ModifyDate',
    ('public', 'SchemewisePortfolio'): 'HoldingDate',
    ('public', 'SectorDivisor'): 'Date',
    ('public', 'SectorDivisor_DUPLICATED'): 'Date',
    ('public', 'SectorIndexList'): 'GenDate',
    ('public', 'SectorIndexList_DUPLICATED'): 'GenDate',
    ('public', 'ShareHolding'): 'SHPDate',
    ('public', 'Splits'): 'DateOfAnnouncement',
    ('public', 'SubIndustryDivisor'): 'Date',
    ('public', 'SubIndustryIndexList'): 'GenDate',
    ('public', 'SubSectorDivisor'): 'Date',
    ('public', 'SubSectorDivisor_DUPLICATED'): 'Date',
    ('public', 'SubSectorIndexList'): 'GenDate',
    ('public', 'SubSectorIndexList_DUPLICATED'): 'GenDate',
    ('public', 'Subsidiaries'): 'ModifiedDate',
    ('public', 'TTM'): 'YearEnding',
    ('public', 'TTM_BACKUP'): 'YearEnding',
    ('public', 'TTM_DUPLICATED'): 'YearEnding',
    ('public', 'TTM_ONWAIT'): 'YearEnding',
    ('public', 'TTM_TEMP'): 'YearEnding',
    ('public', 'index_btt_mapping'): 'ModifiedDate',
    ('public', 'mf_category_mapping'): 'date',
    ('public', 'mf_category_mapping_DUPLICATED'): 'date',
    ('public', 'mf_category_mapping_copy'): 'date',
    ('public', 'mf_ohlc'): 'date',
    ('public', 'mf_ohlc_DUPLICATED'): 'date',
    ('public', 'nse_index_change'): 'date',
    ('public', 'nse_index_change_DUPLICATED'): 'date',
    ('public', 'ohlc_monthly'): 'date',
    ('public', 'ohlc_weekly'): 'date',
    ('public', 'ohlc_weekly_DUPLICATED'): 'date',
    ('public', 'users'): 'ModifiedDate',
    ('public', 'temp_scheme_nav_details'): 'NAVDate',
    ('public', 'temp_schemewiseportfolio'): 'HoldingDate',
    ('mf_analysis', 'ema50_daily'): 'date',
    ('mf_analysis', 'ema50_daily_DUPLICATED'): 'date',
    ('mf_analysis', 'ema50_monthly'): 'date',
    ('mf_analysis', 'ema50_weekly'): 'date',
    ('mf_analysis', 'ema50_weekly_DUPLICATED'): 'date',
    ('mf_analysis', 'indicators'): 'gen_date',
    ('mf_analysis', 'indicators_DUPLICATED'): 'gen_date',
    ('mf_analysis', 'indicators_monthly'): 'gen_date',
    ('mf_analysis', 'indicators_weekly'): 'gen_date',
    ('mf_analysis', 'indicators_weekly_DUPLICATED'): 'gen_date',
    ('mf_analysis', 'market_quality_number'): 'date',
    ('mf_analysis', 'market_quality_number_DUPLICATED'): 'date',
    ('mf_analysis', 'trend_weightage_daily'): 'date',
    ('mf_analysis', 'trend_weightage_daily_DUPLICATED'): 'date',
    ('mf_analysis', 'trend_weightage_monthly'): 'date',
    ('mf_analysis', 'trend_weightage_monthly_DUPLICATED'): 'date',
    ('mf_analysis', 'trend_weightage_weekly'): 'date',
    ('mf_analysis', 'trend_weightage_weekly_DUPLICATED'): 'date',
    ('mf_analysis', 'trends'): 'gen_date',
    ('mf_analysis', 'trends_DUPLICATED'): 'gen_date',
    ('mf_analysis', 'trends_monthly'): 'gen_date',
    ('mf_analysis', 'trends_weekly'): 'gen_date',
    ('mf_analysis', 'trends_weekly_DUPLICATED'): 'gen_date',
    ('logs', 'OHLC'): 'log_date',
    ('logs', 'split_bonus'): 'log_date',
    ('logs', 'report_generation'): 'log_date',
    ('logs', 'insert'): 'log_date',
    ('dash_process', 'Consolidated_EPS'): 'epsdate',
    ('dash_process', 'EPS'): 'epsdate',
    ('dash_process', 'Summary'): 'Date',
    ('dash_process', 'index_off_high'): 'date',
    ('dash_process', 'index_off_high_DUPLICATED'): 'date',
    ('dash_process', 'index_off_low'): 'date',
    ('dash_process', 'index_off_low_DUPLICATED'): 'date',
    ('dash_process', 'index_performance'): 'date',
    ('dash_process', 'index_performance_DUPLICATED'): 'date',
    ('dash_process', 'stock_off_high'): 'date',
    ('dash_process', 'stock_off_high_DUPLICATED'): 'date',
    ('dash_process', 'stock_off_low'): 'date',
    ('dash_process', 'stock_off_low_DUPLICATED'): 'date',
    ('dash_process', 'stock_performance'): 'date',
    ('dash_process', 'stock_performance_DUPLICATED'): 'date',
    ('Reports', 'CombinedRS'): 'GenDate',
    ('Reports', 'CombinedRS_DUPLICATED'): 'GenDate',
    ('Reports', 'Consolidated_EERS'): 'EERSDate',
    ('Reports', 'Consolidated_EPS'): 'EPSDate',
    ('Reports', 'Consolidated_ERS'): 'ERSDate',
    ('Reports', 'EERS'): 'EERSDate',
    ('Reports', 'EPS'): 'EPSDate',
    ('Reports', 'ERS'): 'ERSDate',
    ('Reports', 'FRS-MFRank'): 'Date',
    ('Reports', 'FRS-MFRank_DUPLICATED'): 'Date',
    ('Reports', 'FRS-NAVCategoryAvg'): 'Date',
    ('Reports', 'FRS-NAVCategoryAvg_DUPLICATED'): 'Date',
    ('Reports', 'FRS-NAVRank'): 'Date',
    ('Reports', 'FRS-NAVRank_DUPLICATED'): 'Date',
    ('Reports', 'IRS'): 'GenDate',
    ('Reports', 'IRS_DUPLICATED'): 'GenDate',
    ('Reports', 'NewHighNewLow'): 'Date',
    ('Reports', 'PRS'): 'Date',
    ('Reports', 'PRS_DUPLICATED'): 'Date',
    ('Reports', 'SMR'): 'SMRDate',
    ('Reports', 'SMR_DUPLICATED'): 'SMRDate',
    ('Reports', 'STANDALONE_EERS'): 'EERSDate',
    ('Reports', 'STANDALONE_EPS'): 'EPSDate',
    ('Reports', 'STANDALONE_ERS'): 'ERSDate'
}

insert_date = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

# Insert the data into the table
for (schema_name, table_name), date_column in date_columns.items():
    insert_sql = f"""INSERT INTO schema_table_date_columns (schema_name, table_name, date_column, insert_date) 
                    VALUES ('{schema_name}', '{table_name}', '{date_column}')"""
    cur.execute(insert_sql)
    conn.commit()


In [3]:
from utils.db_helper import DB_Helper
import pandas as pd
from datetime import datetime
from  reports.combined_rank import CombinedRank
import warnings
warnings.filterwarnings("ignore")

db = DB_Helper()
conn = db.db_connect()
cur = conn.cursor()

start_date = '2020-08-05'
end_date = '2025-02-14' 

def delete_combined_rank(date):
    sql = f'DELETE FROM "Reports"."CombinedRS" WHERE "GenDate" = \'{date}\''
    cur.execute(sql)
    conn.commit()
    print(f"Deleted Combined Rank data for {date}")

date_range = pd.date_range(start=start_date, end=end_date)
for curr_date in date_range:
    delete_combined_rank(curr_date.date())
    CombinedRank().current_rank(curr_date, conn,cur)

Deleted Combined Rank data for 2020-08-05
Calculating combined rank for today: 2020-08-05

Getting all 5 ranks for stocks
PRS: CompanyCode          float64
Combined Strength    float64
Value Average        float64
dtype: object
ERS: CompanyCode    float64
Ranking        float64
dtype: object
RRS: CompanyCode    float64
SMR Rank       float64
dtype: object
FRS: CompanyCode    float64
MFRank         float64
dtype: object
IRS: CompanyCode    float64
Rank            object
dtype: object
Calculating combined rank
Merging with background info
Inserting into Table
Deleted Combined Rank data for 2020-08-06
Calculating combined rank for today: 2020-08-06

Getting all 5 ranks for stocks
PRS: CompanyCode          float64
Combined Strength    float64
Value Average        float64
dtype: object
ERS: CompanyCode    float64
Ranking        float64
dtype: object
RRS: CompanyCode    float64
SMR Rank       float64
dtype: object
FRS: CompanyCode    float64
MFRank         float64
dtype: object
IRS: CompanyC

In [1]:
from utils.db_helper import DB_Helper
import pandas as pd
from datetime import datetime
import warnings
warnings.filterwarnings("ignore")
from dash_process import index_offhighlow, index_change

db = DB_Helper()
conn = db.db_connect()
cur = conn.cursor()

start_date = '2024-02-28'
end_date = '2025-02-14'

def delete_index_off_low(date):
    sql = f'DELETE FROM "dash_process"."index_off_low" WHERE "date" = \'{date}\''
    cur.execute(sql)
    conn.commit()
    print(f"Deleted Index Off Low data for {date}")
    
def delete_index_off_high(date):
    sql = f'DELETE FROM "dash_process"."index_off_high" WHERE "date" = \'{date}\''
    cur.execute(sql)
    conn.commit()
    print(f"Deleted Index Off High data for {date}")
    
def delete_index_performance(date):
    sql = f'DELETE FROM "dash_process"."index_performance" WHERE "date" = \'{date}\''
    cur.execute(sql)
    conn.commit()
    print(f"Deleted Index Performance data for {date}")
    
date_range = pd.date_range(start=start_date, end=end_date)
for curr_date in date_range:
    delete_index_off_low(curr_date.date())
    delete_index_off_high(curr_date.date())
    delete_index_performance(curr_date.date())

    try:
        index_offhighlow.main(curr_date,conn, cur)
    except Exception as e:
        print(e, flush=True)
    index_change.main(curr_date)

Deleted Index Off Low data for 2024-02-28
Deleted Index Off High data for 2024-02-28
Deleted Index Performance data for 2024-02-28
prs data for 1 year
index data contain sector, sun and ind
Data which we get it is main
Industry Data with IndexNames
sector Data with SectorIndexNames
Subsector Data with SubSectorIndexNames
industry Data with IndustryIndexNames
Concating all the data for sec, sub , indus OFFHIGH
Concating all the data for sec, sub , indus OFFLOW
Inserting the offhigh cal data to DB
Inserting the offlow cal data to DB
Starting index performance process for date: 2024-02-28 00:00:00

Getting index OHLC data for current date
Getting backdate index OHLC data
Calculating index change values
Inserting index performance data into the DB
Deleted Index Off Low data for 2024-02-29
Deleted Index Off High data for 2024-02-29
Deleted Index Performance data for 2024-02-29
prs data for 1 year
index data contain sector, sun and ind
Data which we get it is main
Industry Data with IndexNam

testing new function to have the same behaviour as the original function which showed day is out of range error.

In [8]:
from datetime import datetime
from dateutil.relativedelta import relativedelta
import pandas as pd
import pandas.io.sql as sqlio
import psycopg2
from utils.db_helper import DB_Helper

# Mock data for testing
mock_data = {
    'DATE': pd.date_range(start='2015-01-01', end='2025-12-31', freq='D'),
    'CLOSE': range(1, len(pd.date_range(start='2015-01-01', end='2025-12-31', freq='D')) + 1)
}
mock_df = pd.DataFrame(mock_data)

# Mock read_sql_query function
def mock_read_sql_query(sql, con):
    return mock_df.query(sql.split('WHERE')[1].strip().replace('"', '').replace(';', ''))

# Original function
def get_index_ohlc_back_original(curr_date, conn):
    """ Function to get OHLC for indexes for backdate. """
    back_date = datetime.date(curr_date.year - 5, curr_date.month, curr_date.day)
    sql = f'SELECT * FROM public."IndexHistory" WHERE "DATE" >= \'{back_date}\' AND "DATE" < \'{curr_date}\';'
    index_ohlc_back = mock_read_sql_query(sql, conn)
    return index_ohlc_back

# Updated function
def get_index_ohlc_back_updated(curr_date, conn):
    """ Function to get OHLC for indexes for backdate. """
    back_date = curr_date - relativedelta(years=5)
    sql = f'SELECT * FROM public."IndexHistory" WHERE "DATE" >= \'{back_date}\' AND "DATE" < \'{curr_date}\';'
    index_ohlc_back = mock_read_sql_query(sql, conn)
    return index_ohlc_back

# Test cases
test_dates = [
    datetime(2020, 2, 29),  # Leap year
    datetime(2021, 2, 28),  # Non-leap year
    datetime(2021, 12, 31), # End of year
    datetime(2021, 1, 31),  # End of month
    datetime(2021, 6, 30),  # End of month
]

# Run test cases
db = DB_Helper()
conn = db.db_connect()

for test_date in test_dates:
    print(f"Testing date: {test_date}")
    try:
        original_result = get_index_ohlc_back_original(test_date, conn)
        print(f"Original function result for {test_date}:\n{original_result.head()}\n")
    except Exception as e:
        print(f"Original function error for {test_date}: {e}")

    try:
        updated_result = get_index_ohlc_back_updated(test_date, conn)
        print(f"Updated function result for {test_date}:\n{updated_result.head()}\n")
    except Exception as e:
        print(f"Updated function error for {test_date}: {e}")

conn.close()

Testing date: 2020-02-29 00:00:00
Original function error for 2020-02-29 00:00:00: descriptor 'date' for 'datetime.datetime' objects doesn't apply to a 'int' object
Updated function error for 2020-02-29 00:00:00: invalid syntax (<unknown>, line 1)
Testing date: 2021-02-28 00:00:00
Original function error for 2021-02-28 00:00:00: descriptor 'date' for 'datetime.datetime' objects doesn't apply to a 'int' object
Updated function error for 2021-02-28 00:00:00: invalid syntax (<unknown>, line 1)
Testing date: 2021-12-31 00:00:00
Original function error for 2021-12-31 00:00:00: descriptor 'date' for 'datetime.datetime' objects doesn't apply to a 'int' object
Updated function error for 2021-12-31 00:00:00: invalid syntax (<unknown>, line 1)
Testing date: 2021-01-31 00:00:00
Original function error for 2021-01-31 00:00:00: descriptor 'date' for 'datetime.datetime' objects doesn't apply to a 'int' object
Updated function error for 2021-01-31 00:00:00: invalid syntax (<unknown>, line 1)
Testing 

In [18]:
from utils.BTTIndex import BTTIndex
from utils.db_helper import DB_Helper
import pandas as pd

db = DB_Helper()
conn = db.db_connect()
cur = conn.cursor() 

curr_date = '2025-03-24'
curr_date = pd.to_datetime(curr_date)

btt_index = BTTIndex()
btt_prevData = btt_index.cal_BTT_divisor_Index(curr_date,conn,cur)



Getting BTT Divisor for date:  2025-03-24
Calculating PrevCLose for btt


c:\BravisaProject\app\utils\BTTIndex.py:51: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  btt_prevData = sqlio.read_sql_query(btt_prevData_sql, con = conn)
c:\BravisaProject\app\utils\BTTIndex.py:58: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  divisor_backdate = sqlio.read_sql_query(divisor_backdate_sql, con = conn)
c:\BravisaProject\app\utils\BTTIndex.py:66: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  btt_divisor_list = sqlio.read_sql_query(sql, con = conn)
c:\BravisaProject\app\utils\BTTIndex.py:78: UserWarning: panda

only OS has changed, no new companies added
      CompanyCode                               CompanyName     NSECode  \
0     10510001.00                        Ashok Leyland Ltd.    ASHOKLEY   
1     10510002.00                         Force Motors Ltd.    FORCEMOT   
2     10510004.00                        Eicher Motors Ltd.   EICHERMOT   
3     10510008.00                          Tata Motors Ltd.  TATAMOTORS   
4     10510008.02                    Tata Technologies Ltd.    TATATECH   
...           ...                                       ...         ...   
1120  17200023.00                           Nxt-Infra Trust   NXT-INFRA   
1121  17200024.00                       Capital Infra Trust    CAPINVIT   
1122  17210022.00              Signatureglobal (India) Ltd.   SIGNATURE   
1123  17240002.00  Unimech Aerospace and Manufacturing Ltd.     UNIMECH   
1124  17530006.00               Enviro Infra Engineers Ltd.        EIEL   

       BSECode          ISIN                           

c:\BravisaProject\app\utils\BTTIndex.py:311: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  prev_close_BTTIndex = sqlio.read_sql_query(prev_close_BTTIndex_sql, con = conn)
c:\BravisaProject\app\utils\BTTIndex.py:404: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pe_back_list = sqlio.read_sql_query(pe_back_sql, con=conn)
c:\BravisaProject\app\utils\BTTIndex.py:577: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  where "DATE" < \''+ str(date) +'\' and "TICKER" = \'BTTIndex\' \
c:\BravisaProject\app\utils\BTTIndex.py:593: UserWar

In [1]:
from utils.db_helper import DB_Helper
import pandas as pd
from mf_analysis.weekly_indicator import WeeklyIndicator

db = DB_Helper()
conn = db.db_connect()

curr_date = '2025-02-14'    

WeeklyIndicator().technical_indicators_weekly(conn, curr_date)

c:\BravisaProject\app\mf_analysis\weekly_indicator.py:52: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  weekly_ohlc_full = sqlio.read_sql_query(weekly_ohlc_full_sql, con=conn)
c:\BravisaProject\app\mf_analysis\weekly_indicator.py:61: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  weekly_trend_prev = sqlio.read_sql_query(


1111
827
weekly_indicator_data:  1
weekly_indicator_data after reset index:  1
weekly_indicator_data:  2
weekly_indicator_data after reset index:  2
weekly_indicator_data:  3
weekly_indicator_data after reset index:  3
weekly_indicator_data:  4
weekly_indicator_data after reset index:  4
weekly_indicator_data:  5
weekly_indicator_data after reset index:  5
weekly_indicator_data:  6
weekly_indicator_data after reset index:  6
weekly_indicator_data:  7
weekly_indicator_data after reset index:  7
weekly_indicator_data:  8
weekly_indicator_data after reset index:  8
weekly_indicator_data:  9
weekly_indicator_data after reset index:  9
weekly_indicator_data:  10
weekly_indicator_data after reset index:  10
weekly_indicator_data:  11
weekly_indicator_data after reset index:  11
weekly_indicator_data:  12
weekly_indicator_data after reset index:  12
weekly_indicator_data:  13
weekly_indicator_data after reset index:  13
weekly_indicator_data:  14
weekly_indicator_data after reset index:  14
w

,company_code,company_name,bse_code,nse_code,open,high,low,close,volume,ema13,ema26,macd,macd_signal,macd_diff,atr,gen_date,ema12,ema50
0,13192325.0,Intellect Design Arena Ltd.,538835.0,INTELLECT,803.00,809.70,693.80,705.30,314838,805.568571,844.929630,-43.895783,-28.259157,-15.636627,91.399286,2025-02-14,801.033846,853.942941
1,13191641.0,Endurance Technologies Ltd.,540153.0,ENDURANCE,1975.00,1990.15,1825.55,1857.60,81559,2071.980000,2167.720370,-109.548063,-55.373613,-54.174450,158.620000,2025-02-14,2058.172308,2141.886471
2,13191103.0,Persistent Systems Ltd.,533179.0,PERSISTENT,6280.00,6284.90,5490.10,5562.70,370089,5841.897143,5604.718519,241.874558,388.326912,-146.452353,515.238571,2025-02-14,5846.593077,5048.661176
3,13190973.0,Redington Ltd.,532805.0,REDINGTON,233.00,248.00,222.50,245.26,11046406,222.348571,211.843333,11.907436,6.165487,5.741949,19.491429,2025-02-14,223.750769,203.523529
4,13190705.0,Coforge Ltd.,532541.0,COFORGE,8550.00,8550.00,7676.00,7793.35,472045,8333.161429,7931.859259,413.309972,639.869994,-226.560023,676.873571,2025-02-14,8345.169231,7197.298627
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
659,11621009.0,Oberoi Realty Ltd.,533273.0,OBEROIRLTY,1812.10,1820.00,1547.80,1570.65,710925,1835.747143,1854.862963,-28.750655,53.833869,-82.584524,164.864286,2025-02-14,1826.112308,1743.053137
660,11580005.0,Kansai Nerolac Paints Ltd.,500165.0,KANSAINER,250.22,257.50,226.75,231.40,203186,250.300000,263.048148,-14.073533,-11.030707,-3.042826,18.047143,2025-02-14,248.974615,279.179804
661,10760016.0,Avenue Supermarts Ltd.,540376.0,DMART,3756.60,3813.00,3585.00,3682.65,286886,3763.547143,3973.687037,-228.387806,-230.805561,2.417755,304.282143,2025-02-14,3745.299231,4084.603725
662,11120007.0,Nestle India Ltd.,500790.0,NESTLEIND,2235.00,2252.50,2170.05,2195.85,960861,2241.450000,2309.729630,-73.391168,-78.438234,5.047066,103.890714,2025-02-14,2236.338462,2585.419216
